## Description:
这个笔记本主要是先把YoutubeDNN召回跑起来， 采用的数据集是data_process下面的train_data.csv， 这个是用户历史行为数据+用户画像数据， 模型的话先调用包实现， 这里面主要完成的步骤是：
1. 导入数据， 划分出测试集来， 最后一天用户id进行评估
2. 用训练集的数据构造YoutubeDNN的数据集出来， 这里采用滑窗的方式构造， 但由于有些序列很长，尝试通过滑动步长优化
3. 构造YoutubeDNN的输入， 由于是调用deepmatch的YoutubeDNN，所以需要把输入特殊构造
4. 建立YoutubeDNN模型训练
5. 从YoutubeDNN中拿到用户embedding和doc的embedding，保存起来
6. 用第11天的测试集，进行评估，看看YoutubeDNN的召回效果

In [1]:
import os
import time
import pickle
import random
from datetime import datetime
import collections

import numpy as np
import pandas as pd

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder

# 从工具包里导入工具函数
from utils import gen_data_set, gen_model_input, train_youtube_model
from utils import get_embeddings, get_youtube_recall_res

import warnings
warnings.filterwarnings('ignore')
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # 1显示所有信息， 2显示error和warnings, 3显示error


ModuleNotFoundError: No module named 'deepmatch'

### 1.导入数据，划分数据集

In [ ]:
data_path = "data_process"
data = pd.read_csv(os.path.join(data_path,"train_data.csv"), index_col=0,parse_dates=["expo_time"])

In [ ]:
data.shape

In [ ]:
latest_time = data['expo_time'].max()

# 添加example_age字段
# 沿用 example_age的定义 从最近时间 - 曝光的时间
data["example_age"] = (pd.to_datetime(latest_time) - data['expo_time'])
# 转成小时的形式 上面两式子相减是pandas的timedelta类型， 只有days和seconds属性
data["example_age"] = data["example_age"].apply(lambda x : x.days*24 + x.seconds // 3600)

minmax = MinMaxScaler() # 归一化
data["example_age"] = minmax.fit_transform(data["example_age"].values.reshape(-1,1))

In [ ]:
data.head()

In [ ]:
# 选择出需要用到的列
use_cols = ['user_id', 'article_id','expo_time','net_status','exop_position','duration','device','city','age','gender','example_age','click']
data_new = data[use_cols]

In [ ]:
data_new['exop_position'].unique().shape

## 划分测试集和训练集
* 训练集， 每个用户的历史点击，去掉最后一次
* 测试集， 每个用户的最后一次点击

In [ ]:
# 按照用户分组，然后把最后一个item拿出来
click_df = data_new[data_new['click']==1]

In [ ]:
click_df.head()

In [ ]:
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id', 'expo_time'])
    click_last_df = all_click.groupby('user_id').tail(1)
    
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

In [ ]:
user_click_hist_df, user_click_last_df = get_hist_and_last_click(click_df)

## 2. YoutubeDNN召回

In [ ]:
def youtubednn_recall(data, topk=200, embedding_dim=8, his_seq_maxlen=50, negsample=0,
                      batch_size=64, epochs=1, verbose=1, validation_split=0.0):
    """通过YouTubeDNN模型，计算用户向量和文章向量
    param: data: 用户日志数据
    topk: 对于每个用户，召回多少篇文章
    """
    user_id_raw = data[['user_id']].drop_duplicates('user_id')
    doc_id_raw = data[['article_id']].drop_duplicates('article_id')
    
    # 类别数据编码   
    base_features = ['user_id', 'article_id', 'city', 'age', 'gender']
    feature_max_idx = {}
    for f in base_features:
        lbe = LabelEncoder()
        data[f] = lbe.fit_transform(data[f])
        feature_max_idx[f] = data[f].max() + 1
        
    # 构建用户id词典和doc的id词典，方便从用户idx找到原始的id
    user_id_enc = data[['user_id']].drop_duplicates('user_id')
    doc_id_enc = data[['article_id']].drop_duplicates('article_id')
    user_idx_2_rawid = dict(zip(user_id_enc['user_id'], user_id_raw['user_id']))
    doc_idx_2_rawid = dict(zip(doc_id_enc['article_id'], doc_id_raw['article_id']))
    
    # 保存下每篇文章的被点击数量， 方便后面高热文章的打压
    doc_clicked_count_df = data.groupby('article_id')['click'].apply(lambda x: x.count()).reset_index()
    doc_clicked_count_dict = dict(zip(doc_clicked_count_df['article_id'], doc_clicked_count_df['click']))

    train_set, test_set = gen_data_set(data, doc_clicked_count_dict, negsample, control_users=False)
    
    # 构造youtubeDNN模型的输入
    train_model_input, train_label = gen_model_input(train_set, his_seq_maxlen)
    test_model_input, test_label = gen_model_input(test_set, his_seq_maxlen)
    
    # 构建模型并完成训练
    model = train_youtube_model(train_model_input, train_label, embedding_dim, feature_max_idx, his_seq_maxlen, batch_size, epochs, verbose, validation_split)
    
    # 获得用户embedding和doc的embedding， 并进行保存
    user_embs, doc_embs = get_embeddings(model, test_model_input, user_idx_2_rawid, doc_idx_2_rawid)
    
    # 对每个用户，拿到召回结果并返回回来
    user_recall_doc_dict = get_youtube_recall_res(user_embs, doc_embs, user_idx_2_rawid, doc_idx_2_rawid, topk)
    
    return user_recall_doc_dict

In [ ]:
user_recall_doc_dict = youtubednn_recall(user_click_hist_df, negsample=3)